# DTSA 5512: Introduction to Generative AI

## Course Overview and Quick Reference Guide

This notebook serves as a comprehensive overview and quick reference guide for the key concepts, techniques, and implementations covered in this course.

### Course Objectives
- Understanding generative AI principles
- Implementing various generative models
- Working with transformers and LLMs
- Applying generative AI to real problems

In [ ]:
# Import common libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

# Display settings
%matplotlib inline
plt.style.use('seaborn')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Week 1: Introduction to Generative AI

### Key Concepts
- 

### Important Terms
- 

### Code Examples

In [ ]:
class SimpleAutoencoder(nn.Module):
    """Simple autoencoder implementation"""
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

## Week 2: Variational Autoencoders

### Key Concepts
- 

### Important Components
- 

### Code Examples

In [ ]:
class VAE(nn.Module):
    """Variational Autoencoder implementation"""
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_var = nn.Linear(128, latent_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_var(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

## Week 3: Generative Adversarial Networks

### Key Concepts
- 

### Important Components
- 

### Code Examples

In [ ]:
class Generator(nn.Module):
    """Simple GAN Generator"""
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Tanh()
        )
        
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    """Simple GAN Discriminator"""
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

## Week 4: Transformers and Language Models

### Key Concepts
- 

### Important Components
- 

### Code Examples

In [ ]:
def load_language_model(model_name="gpt2"):
    """Load a pre-trained language model"""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

def generate_text(prompt, model, tokenizer, max_length=50):
    """Generate text using a language model"""
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## Week 5: Diffusion Models

### Key Concepts
- 

### Important Components
- 

### Code Examples

In [ ]:
class SimpleDiffusion:
    """Simple diffusion model implementation"""
    def __init__(self, n_steps=1000, beta_start=1e-4, beta_end=0.02):
        self.n_steps = n_steps
        self.beta = torch.linspace(beta_start, beta_end, n_steps)
        self.alpha = 1 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        
    def forward_diffusion(self, x0, t):
        """Forward diffusion process"""
        noise = torch.randn_like(x0)
        alpha_t = self.alpha_bar[t]
        mean = torch.sqrt(alpha_t) * x0
        var = torch.sqrt(1 - alpha_t)
        return mean + var * noise, noise

## Week 6: Advanced Architectures

### Key Concepts
- 

### Important Models
- 

### Code Examples

In [ ]:
class StyleGANBlock(nn.Module):
    """Simple StyleGAN-like block"""
    def __init__(self, in_channels, out_channels, style_dim):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.noise_weight = nn.Parameter(torch.zeros(1, out_channels, 1, 1))
        self.activation = nn.LeakyReLU(0.2)
        self.style_mod = nn.Linear(style_dim, out_channels * 2)
        
    def forward(self, x, style):
        x = self.conv(x)
        noise = torch.randn(x.shape[0], 1, x.shape[2], x.shape[3], device=x.device)
        x = x + self.noise_weight * noise
        
        style = self.style_mod(style).unsqueeze(2).unsqueeze(3)
        gamma, beta = style.chunk(2, dim=1)
        x = gamma * x + beta
        
        return self.activation(x)

## Week 7: Training and Optimization

### Key Concepts
- 

### Important Techniques
- 

### Code Examples

In [ ]:
def train_gan(generator, discriminator, dataloader, n_epochs=100):
    """Basic GAN training loop"""
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)
    criterion = nn.BCELoss()
    
    for epoch in range(n_epochs):
        for real_data in dataloader:
            batch_size = real_data.size(0)
            real_label = torch.ones(batch_size, 1)
            fake_label = torch.zeros(batch_size, 1)
            
            # Train Discriminator
            d_optimizer.zero_grad()
            label = ((real_label + fake_label) / 2).to(device)
            output = discriminator(real_data.to(device))
            d_loss_real = criterion(output, real_label.to(device))
            
            z = torch.randn(batch_size, generator.latent_dim).to(device)
            fake_data = generator(z)
            output = discriminator(fake_data.detach())
            d_loss_fake = criterion(output, fake_label.to(device))
            
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # Train Generator
            g_optimizer.zero_grad()
            output = discriminator(fake_data)
            g_loss = criterion(output, real_label.to(device))
            g_loss.backward()
            g_optimizer.step()

## Week 8: Applications and Future Directions

### Key Concepts
- 

### Important Applications
- 

### Code Examples

In [ ]:
def image_to_image_translation(source_image, style_image, model):
    """Simple image-to-image translation example"""
    # Implement based on specific model architecture
    pass

def text_to_image_generation(text_prompt, model):
    """Simple text-to-image generation example"""
    # Implement based on specific model architecture
    pass

## Additional Resources and References

### Useful Libraries
- PyTorch: Deep learning framework
- Transformers: NLP models
- Diffusers: Diffusion models
- Lightning: Training framework

### External Links
- Course materials
- Research papers
- Model implementations

### Personal Notes
- Key architectures
- Training tips
- Best practices